In [1]:
%matplotlib inline

In [2]:
import json

import numpy as np
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

from utils import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
MODELS_BASEDIR = 'http://www.platform.ai/models/'
IMAGENET_CLASSES_FILE = 'imagenet_class_index.json'

fpath = get_file(IMAGENET_CLASSES_FILE, MODELS_BASEDIR + IMAGENET_CLASSES_FILE, cache_subdir='models')
with open(fpath) as f: 
    class_dict = json.load(f)

# Convert dictionary with string indexes into an array
classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

In [4]:
def ConvBlock(layers, model, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

In [5]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    #model.add(Dropout(0.5))

In [6]:
def vgg_preprocess(x):
    vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

In [7]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))

    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    
    VGG16_WEIGHTS_FILE = 'vgg16.h5'
    fpath = get_file(VGG16_WEIGHTS_FILE, MODELS_BASEDIR + VGG16_WEIGHTS_FILE, cache_subdir='models')
    model.load_weights(fpath)
    
    return model

In [8]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=4, class_mode='categorical'):
    return gen.flow_from_directory(dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [9]:
model = VGG_16()

In [10]:
DATA_DIR = "data/dogs-vs-cats-redux-kernels-edition/"

In [11]:
batch_size = 64
train_batches = get_batches(DATA_DIR + 'train', batch_size = batch_size)
val_batches = get_batches(DATA_DIR + 'valid', batch_size = batch_size)
#imgs, labels = next(batches)

#plots(imgs, titles = labels)

Found 22980 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
train_batches.nb_sample

22980

In [13]:
val_data = get_data(DATA_DIR + 'valid')
train_data = get_data(DATA_DIR + 'train')

Found 2000 images belonging to 2 classes.
Found 22980 images belonging to 2 classes.


In [14]:
print(train_data.shape)
print(val_data.shape)

(22980, 3, 224, 224)
(2000, 3, 224, 224)


In [15]:
val_classes = val_batches.classes
train_classes = train_batches.classes
val_labels = onehot(val_classes)
train_labels = onehot(train_classes)

In [16]:
print(train_labels.shape)
print(val_labels.shape)

(22980, 2)
(2000, 2)


In [17]:
save_array(DATA_DIR + 'train_data.bc', train_data)
save_array(DATA_DIR + 'valid_data.bc', val_data)

In [18]:
# Skip this step for now. 
#train_data = load_array(DATA_DIR + 'train_data.bc')
#val_data = load_array(DATA_DIR + 'valid_data.bc')

In [18]:
model.pop()
for layer in model.layers:
    layer.trainable=False
model.add(Dense(train_batches.nb_class, activation='softmax', input_shape=(1000,)))
model.compile(optimizer=RMSprop(lr=0.1),
        loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________


OSError: [Errno 12] Cannot allocate memory

## Train using the saved training data

In [20]:
batch_size = 8

In [21]:
gen = image.ImageDataGenerator()
train_data_batches = gen.flow(train_data, train_labels, batch_size = batch_size, shuffle = True)
val_data_batches = gen.flow(val_data, val_labels, batch_size = batch_size, shuffle = False)

In [22]:
train_batches.N

22980

In [36]:
model.fit_generator(train_data_batches, samples_per_epoch=train_data_batches.N, nb_epoch=1,
                validation_data=val_data_batches, nb_val_samples=val_data_batches.N)

Epoch 1/2
22980/22980 [==============================] - 611s - loss: 0.5177 - acc: 0.9671 - val_loss: 0.3748 - val_acc: 0.9765
Epoch 2/2
22980/22980 [==============================] - 611s - loss: 0.5220 - acc: 0.9671 - val_loss: 0.4914 - val_acc: 0.9690
